In [1]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import DebugStatus
from benchmark_consts import *

In [6]:
options = DebugOptions()
# remember to set this!
options.is_verus = True

round1 = BenchViewer(UNSTABLE_VERUS, options)

[WARN] [proof] no proofs available 
[WARN] [proof] no proofs available 
[WARN] [init] 8d058577cd trace slow unknown, fallback to fast_fail 
[WARN] [proof] no proofs available 


In [7]:
round1.status.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| DebugStatus.FIX_FOUND     |      50 | 71.43 %      |
| DebugStatus.FIX_NOT_FOUND |      17 | 24.29 %      |
| DebugStatus.NO_PROOF      |       3 | 4.29 %       |
| total                     |      70 | 100.00 %     |


In [8]:
discount = [
    ("08719204aa", "4b9e624e"),
    ("81b159a977", "fbe192f3"),
    ("2556504d15", "7ab783b0"),
    ("6540edb885", "5e546c11"),
    ("82449bd2e8", "4c612f9b"),
    ("a1b4b3fd4a", "4e444e51"),
    ("55b4af82a4", "02d50371"),
    ("2556504d15", "0e424555"),
    ("4d2d4fbc20", "5a3b4208"),
    ("82449bd2e8", "905f3174"),
    ("2045867a58", "7ec12ea5"),
    ("55b4af82a4", "781157b6"),
    ("68ac429152", "75d801e3"),
]

In [9]:
def fixable_after_discount(dbg):
    report =dbg.report
    name_hash = dbg.name_hash
    fixable = False
    for edit_path in report.stabilized['edit_path']:
        edit_hash = os.path.basename(edit_path).split('.')[0]
        if (name_hash, edit_hash) in discount:
            continue
        fixable = True
    return fixable

In [ ]:
from utils.analysis_utils import fmt_percent

bench_total = len(round1.status.tally)

to_cases = set()
rd1_to_fixable = set()
ff_cases = set()
rd1_ff_fixable = set()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

for q in round1.status.tally:
    dbg = round1[q]
    if dbg.mode == DbgMode.TIMEOUT:
        to_cases.add(q)
        if q in round1.status[DebugStatus.FIX_FOUND] and fixable_after_discount(dbg):
            rd1_to_fixable.add(q)
    elif dbg.mode == DbgMode.FAST_FAIL:
        ff_cases.add(q)
        if q in round1.status[DebugStatus.FIX_FOUND] and fixable_after_discount(dbg):
            rd1_ff_fixable.add(q)
    else:
        print("unexpected mode", dbg.mode)
        assert False

print("TOs:", print_break_down(len(to_cases), bench_total))
rd1_to_no_proof = to_cases & round1.status[DebugStatus.NO_PROOF].items
rd1_to_no_fix = to_cases & round1.status[DebugStatus.FIX_NOT_FOUND].items
print("\t- 1-no-proof:", print_break_down(len(rd1_to_no_proof), len(to_cases)))
print("\t- 1-fixable:", print_break_down(len(rd1_to_fixable), len(to_cases)))
print("\t- 1-unfixable:", print_break_down(len(rd1_to_no_fix), len(to_cases)))

print("FFs:", print_break_down(len(ff_cases), bench_total))
rd1_ff_no_proof = ff_cases & round1.status[DebugStatus.NO_PROOF].items
rd1_ff_no_fix = ff_cases & round1.status[DebugStatus.FIX_NOT_FOUND].items
print("\t- 1-no-proof:", print_break_down(len(rd1_ff_no_proof), len(ff_cases)))
print("\t- 1-fixable:", print_break_down(len(rd1_ff_fixable), len(ff_cases)))
print("\t- 1-unfixable:", print_break_down(len(rd1_ff_no_fix), len(ff_cases)))

print("")
print("1-fixable:", print_break_down(len(rd1_to_fixable | rd1_ff_fixable), bench_total))

TOs: 39/70 (55.7%)
	- 1-no-proof: 2/39 (5.1%)
	- 1-fixable: 25/39 (64.1%)
	- 1-unfixable: 9/39 (23.1%)
FFs: 31/70 (44.3%)
	- 1-no-proof: 1/31 (3.2%)
	- 1-fixable: 22/31 (71.0%)
	- 1-unfixable: 8/31 (25.8%)

1-fixable: 47/70 (67.1%)


In [ ]:
# for Verus, we bundle and re-test the fixable cases

for query in rd1_to_fixable | rd1_ff_fixable:
    report = round1[query].report
    name_hash = round1[query].name_hash

    for edit_path in report.stabilized['edit_path']:
        edit_query = os.path.basename(edit_path)
        bundle_path = f"data/projs/verus_singleton_bundle/base.z3/{name_hash}.{edit_query}"

In [ ]:
for q in round1.status[DebugStatus.NO_PROOF]:
    print(q)

In [12]:
for q in round1.status[DebugStatus.NOT_CREATED]:
    print("./src/debugger3.py -m auto -i", round1[q].name_hash, "--create-project")

In [11]:
from utils.system_utils import list_smt2_files

for q in round1.status[DebugStatus.UNFILTERED]:
    assert len(list_smt2_files(round1[q].strainer.filter_dir)) != 0
    print("./src/make_spaghet.py --verus --local -i", round1[q].strainer.filter_dir)

for q in round1.status[DebugStatus.NOT_TESTED]:
    assert len(list_smt2_files(round1[q].strainer.test_dir)) != 0
    print("./src/make_spaghet.py --verus --local -i", round1[q].strainer.test_dir)

In [20]:
from debugger.edit_info import EditAction, EditInfo

skolem_failed = []

for q in rd1_ff_no_fix:
    dbg = round1[q]
    qname = dbg.editor.choose_qanme_to_skolemize()

    if qname is None:
        skolem_failed.append(q)
        continue

    ei = EditInfo(VERUS_SKOLEM_DIR, {qname: EditAction.SKOLEMIZE})
    edit_hash = ei.get_id()
    name_hash = dbg.name_hash

    query_path = f"{VERUS_SKOLEM_DIR}/{name_hash}.{edit_hash}.smt2"

    if os.path.exists(query_path):
        continue

    if not dbg.editor.edit_by_qname(qname, EditAction.SKOLEMIZE):
        skolem_failed.append(q)
        continue

    if not dbg.editor.save(query_path):
        skolem_failed.append(q)
        continue

In [22]:
[round1[q].name_hash for q in rd1_to_no_fix]

['621abdd996',
 'aad03b9358',
 '199ba4594c',
 '93004876e4',
 '65d90190b4',
 'fa2fdaf6e0',
 '6937ca47cd',
 '8c0bbf67d0',
 '66c9bdf69f']